# TriviaQA

In [ ]:
!git clone https://github.com/aaronkossler/triviaqa.git

## Import Dataset

In [ ]:
%%bash

pip install datasets

In [ ]:
from datasets import load_dataset

trivia_qa_wikipedia = load_dataset('trivia_qa', name="rc.wikipedia")

In [11]:
import sys
import json
import pandas as pd

# sys.path.append("../")

train = pd.read_json('../data/wikipedia-train.json', encoding='utf-8')

In [17]:
len(train[train["Split"] == "train"])

61888

In [4]:
test = trivia_qa_wikipedia["validation"]

In [5]:
import json
import os

# Convert the evaluation set to the desired format
data = []
for item in test:
    answer = {
        "Aliases": item["answer"]["aliases"],
        "MatchedWikiEntityName": item["answer"]["matched_wiki_entity_name"],
        "NormalizedAliases": item["answer"]["normalized_aliases"],
        "NormalizedMatchedWikiEntityName": item["answer"]["normalized_matched_wiki_entity_name"],
        "NormalizedValue": item["answer"]["normalized_value"],
        "Type": item["answer"]["type"],
        "Value": item["answer"]["value"],
    }
    entity_pages = [
        {
            "DocSource": item["entity_pages"]["doc_source"][index],
            "Filename": item["entity_pages"]["filename"][index],
            "Title": item["entity_pages"]["title"][index],
        }
        for index in range(len(item["entity_pages"]["filename"]))
    ]
    question = item["question"]
    question_id = item["question_id"]
    question_source = item["question_source"]
    search_results = []
    data_item = {
        "Answer": answer,
        "EntityPages": entity_pages,
        "Question": question,
        "QuestionId": question_id,
        "QuestionSource": question_source,
        "SearchResults": search_results,
    }
    data.append(data_item)

output = {
    "Data": data,
    "Domain": "Wikipedia",
    "VerifiedEval": False,
    "Version": 1.0,
}

# Write the output to a JSON file
if not os.path.exists("triviaqa/sets"):
    os.makedirs("triviaqa/sets")

with open("triviaqa/sets/evaluation.json", "w") as f:
    json.dump(output, f)

## Preprocessing

## Model Training

In [ ]:
!pip install transformers
!pip install evaluate
!pip install rouge


import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

### Hyperparameters

In [ ]:
TOKENIZER = T5TokenizerFast.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
MODEL.to("cuda")
OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)
Q_LEN = 256   # Question Length
T_LEN = 32    # Target Length
BATCH_SIZE = 8
DEVICE = "cuda:0"

### Extracting context, question, and answers from the dataset

In [42]:
import re

def prepare_data(data):
    articles = []

    for item in data:
        question = item["Question"]
        answer = item["Answer"]["Value"]

        texts = []
        for pages in item["EntityPages"]:
            filename = pages["Filename"]
            filename = re.sub("[\:?*\"]", "_", filename)
            text = file=open(f"../evidence/wikipedia/{filename}", mode="r", encoding="utf-8").read()
            texts.append(text)
        context = " ".join(texts)

        inputs = {"context": context, "question": question, "answer": answer}
        articles.append(inputs)

    return articles

In [27]:
train["Data"].iloc[0]

{'Answer': {'Aliases': ['Park Grove (1895)',
   'York UA',
   'Yorkish',
   'UN/LOCODE:GBYRK',
   'York, UK',
   'Eoforwic',
   'Park Grove School',
   'York Ham',
   'The weather in York',
   'City of York',
   'York, England',
   'York, Yorkshire',
   'York ham',
   'County Borough of York',
   'YORK',
   'Eoferwic',
   'Park Grove Primary School',
   'York, North Yorkshire',
   'Yoisk',
   'York',
   'York (England)'],
  'MatchedWikiEntityName': 'York',
  'NormalizedAliases': ['york yorkshire',
   'eoferwic',
   'park grove primary school',
   'park grove school',
   'weather in york',
   'park grove 1895',
   'eoforwic',
   'county borough of york',
   'york uk',
   'un locode gbyrk',
   'city of york',
   'york england',
   'york ua',
   'york ham',
   'york',
   'yorkish',
   'yoisk',
   'york north yorkshire'],
  'NormalizedMatchedWikiEntityName': 'york',
  'NormalizedValue': 'york',
  'Type': 'WikipediaEntity',
  'Value': 'York'},
 'EntityPages': [{'DocSource': 'TagMe',
   'Fil

In [43]:
data = prepare_data(train["Data"])

# Create a Dataframe
data = pd.DataFrame(data)

In [10]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]

        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)

        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100

        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [11]:
# Dataloader
val_data, train_data = train_test_split(data, shuffle=False, train_size=7900)
# val_data, train_data = train_test_split(data[:1000], shuffle=False, train_size=100)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [ ]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(4):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1

    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1

    print(f"{epoch+1}/{2} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

    MODEL.save_pretrained(f"qa_model-to-batch-{epoch}")
    TOKENIZER.save_pretrained(f"qa_tokenizer-to-batch-{epoch}")

Training batches:   8%|▊         | 548/6749 [09:35<1:39:21,  1.04it/s]

In [ ]:
MODEL.save_pretrained("qa_model")
TOKENIZER.save_pretrained("qa_tokenizer")

# Saved files
"""('qa_tokenizer/tokenizer_config.json',
 'qa_tokenizer/special_tokens_map.json',
 'qa_tokenizer/spiece.model',
'qa_tokenizer/added_tokens.json',
'qa_tokenizer/tokenizer.json')"""

In [ ]:
def predict_answer(context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_answer:
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer],
                            references=[ref_answer])

        print("Context: \n", context)
        print("\n")
        print("Question: \n", question)
        return {
            "Reference Answer: ": ref_answer,
            "Predicted Answer: ": predicted_answer,
            "BLEU Score: ": score
        }
    else:
        return predicted_answer

## Model Prediction

In [47]:
test = pd.read_json('../data/verified-wikipedia-dev.json', encoding='utf-8')
test_data = test["Data"]

In [46]:
test.head(5)

,Data,Domain,Split,VerifiedEval,Version
0,"{'Answer': {'Aliases': ['The Swiss Miss', 'Mar...",Wikipedia,dev,True,1
1,"{'Answer': {'Aliases': ['Baby Buggy', 'Baby bu...",Wikipedia,dev,True,1
2,"{'Answer': {'Aliases': ['Iwerks, Ub', 'Ub Iwer...",Wikipedia,dev,True,1
3,"{'Answer': {'Aliases': ['ISO 3166-1:CH', 'Svis...",Wikipedia,dev,True,1
4,"{'Answer': {'Aliases': ['Receptors, lh', 'LH s...",Wikipedia,dev,True,1


In [48]:
# Testing with first Entry
entry = test_data[0]
question = entry["Question"]
answer = entry["Answer"]["Value"]

texts = []
for pages in entry["EntityPages"]:
    filename = pages["Filename"]
    filename = re.sub("[\:?*\"]", "_", filename)
    text = file=open(f"../evidence/wikipedia/{filename}", mode="r", encoding="utf-8").read()
    texts.append(text)
context = " ".join(texts)

# predict_answer(context, question, answer)

In [49]:
context

'Tennis is a racket sport that can be played individually against a single opponent (singles) or between two teams of two players each (doubles). Each player uses a tennis racket that is strung with cord to strike a hollow rubber ball covered with felt over or around a net and into the opponent\'s court. The object of the game is to play the ball in such a way that the opponent is not able to play a valid return. The player who is unable to return the ball will not gain a point, while the opposite player will.\n\nTennis is an Olympic sport and is played at all levels of society and at all ages. The sport can be played by anyone who can hold a racket, including wheelchair users. The modern game of tennis originated in Birmingham, England, in the late 19th century as "lawn tennis".  It had close connections both to various field ("lawn") games such as croquet and bowls as well as to the older racket sport of real tennis. During most of the 19th century, in fact, the term "tennis" referre

In [ ]:
predictions = {}
for entry in test_data:
    question = entry["Question"]
    answer = entry["Answer"]["Value"]
    
    texts = []
    for pages in entry["EntityPages"]:
        filename = pages["Filename"]
        filename = re.sub("[\:?*\"]", "_", filename)
        text = file=open(f"../evidence/wikipedia/{filename}", mode="r", encoding="utf-8").read()
        texts.append(text)
    context = " ".join(texts)
    predictions[entry["question_id"]] = predict_answer(context, question)

In [ ]:
if not os.path.exists("triviaqa/predictions"):
        os.makedirs("triviaqa/predictions")

# Convert the dictionary to a JSON string
json_string = json.dumps(predictions)

# Write the JSON string to a file
with open("triviaqa/predictions/t5_predictions.json", "w") as f:
    f.write(json_string)

## Evaluation

In [ ]:
import sys
sys.path.append("./triviaqa")

In [ ]:
from triviaqa.evaluation.triviaqa_evaluation import evaluate_triviaqa
from triviaqa.utils.dataset_utils import *
from triviaqa.utils.utils import read_json

In [ ]:
dataset_file = 'triviaqa/sets/evaluation.json'
prediction_file = 'triviaqa/predictions/t5_predictions.json'

expected_version = 1.0
dataset_json = read_triviaqa_data(dataset_file)
if dataset_json['Version'] != expected_version:
    print('Evaluation expects v-{} , but got dataset with v-{}'.format(expected_version,dataset_json['Version']),
          file=sys.stderr)
key_to_ground_truth = get_key_to_ground_truth(dataset_json)
predictions = read_json(prediction_file)
eval_dict = evaluate_triviaqa(key_to_ground_truth, predictions)

In [ ]:
print(eval_dict)

{'exact_match': 47.59164268735143, 'f1': 55.40320738832585, 'common': 7993, 'denominator': 7993, 'pred_len': 7993, 'gold_len': 7993}
